# Calculate functional brain connectivity and certain graph theory metrics (currently limited to weighted degree centrality):

### Check https://pubmed.ncbi.nlm.nih.gov/24238796/ , https://www.sciencedirect.com/science/article/pii/S0924977X10000684 , https://pubmed.ncbi.nlm.nih.gov/20817103/ and https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7086233/

In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import nibabel as nib
from nilearn.input_data import NiftiMasker

from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

import graph_tool.all as gt

import csv
import pickle

In [2]:
# Load an image from a single brain:
def img_load(folder, img):
    img_path = os.path.join(folder, img)
    fmri_image = nib.load(img_path)
    #print(fmri_image.shape)
    return fmri_image

In [3]:
# Load a resampled, binarised GM mask
def mask_img_load(folder, img):
    img_path = os.path.join(folder, img)
    mask = nib.load(img_path)
    return mask

In [4]:
# Extract the timeseries from a single session
def extract_time_series(fmri_image, mask):
    brain_masker = NiftiMasker(mask, memory='nilearn_cache', memory_level=1, verbose=0)
    brain_time_series = brain_masker.fit_transform(fmri_image)
    return brain_time_series, brain_masker

In [5]:
# Compute cosine similarity (N.B. same as Pearson correlation when data is centered)
def cos_sim_func(time_series):
    cos_sim = cosine_similarity(time_series.T, time_series.T)
    return cos_sim

In [6]:
# Threshold the adjacency matrix
def thresh_mat(adjacency_matrix):
    # threshold at r > 0.25
    adjacency_matrix[adjacency_matrix < 0.25] = 0
    # fill diagonal with zeroes
    np.fill_diagonal(adjacency_matrix, 0)
    return adjacency_matrix

In [7]:
# https://carlonicolini.github.io/sections/science/2018/09/12/weighted-graph-from-adjacency-matrix-in-graph-tool.html
# To create an undirected, weighted graph with graph-tool
def to_graph_tool(adj):
    g = gt.Graph(directed=False)
    edge_weights = g.new_edge_property('double')
    g.edge_properties['weight'] = edge_weights
    # Set the lower triangle of the adjacency matrix and the diagonal to 0
    nnz = np.nonzero(np.triu(adj,1))
    # Get the number of edges (i.e. non-zero values)
    nedges = len(nnz[0])
    # Create the edge value list
    g.add_edge_list(
        # Create rows of THREE values
        np.hstack(
        [
        # Transpose nnz so that you have TWO values in each row, where
        # the first is the row index and the second is the column index
        # of this particular edge
        np.transpose(nnz),
        # Get the 3RD values for each row, i.e. the edge weight
        np.reshape(adj[nnz],(nedges,1))
        ]
        ),
    # If given, eprops should specify an iterable containing edge property maps that will be filled with the remaining values at each row, if there are more than two.
    eprops=[edge_weights])
    return g

In [8]:
# Get n of degrees for each voxel (first option) or get the weighted sum for each voxel (second option)
def calc_dc(adjacency_matrix):
    g = to_graph_tool(adjacency_matrix)
    
    #Un-weighted
    #dc = g.get_total_degrees([i for i in range(adjacency_matrix.shape[0])])
    
    #Weighted
    weights = g.degree_property_map("total", g.edge_properties['weight'])
    dc_list = [weights[i] for i in range(adjacency_matrix.shape[0])]
    dc = np.array(dc_list)
    
    return dc

In [9]:
# Z-score
#def z_score_dc(dc_array):
#    dc = dc_array.reshape(-1, 1)
#    scaler = StandardScaler()
#    dc_scaled = scaler.fit_transform(dc)
#    return dc_scaled

In [10]:
# Transform the FC or DC array back into a NIfTI volume and save
def array_to_nifti(brain_masker,array):
    img = brain_masker.inverse_transform(array.T)
    return img

## Run the DC analysis on the whole dataset

In [12]:
# To calculate functional connectivity and degree centrality from an adjacency matrix
def calc_dc_auto(fMRIroot, fMRI_txt, MASKroot, GM_mask):
    
    # transform the txt file into a Python list... of lists!
    fmri_list = []
    with open(fMRI_txt, newline='') as inputfile:
        for row in csv.reader(inputfile):
            fmri_list.append(row)
    
    for i in range(len(fmri_list)):
        # load an image from a single brain:
        fmri_image = img_load(fMRIroot, fmri_list[i][0])
        print("Participant " + fmri_list[i][0])
        print("The image dimensions are: " + str(fmri_image.shape))
        
        # load the GM mask
        gm_mask = mask_img_load(MASKroot, GM_mask)
        print("GM mask loaded.")
        
        # create a NiftiMasker object and extract the time series
        brain_time_series, brain_masker = extract_time_series(fmri_image, gm_mask)
        print("Time series extracted.")
        
        # calculate the correlations between each pair of voxels
        cos_sim = cos_sim_func(brain_time_series)
        print("Correlations calculated.")
        
        # threshold the matrix
        adjacency_matrix = thresh_mat(cos_sim)
        print("Adjacency matrix thresholded.")
        
        # calculate the degree centrality (DC)
        dc = calc_dc(adjacency_matrix)
        print("Raw wDC values calcualted.")
        
        # convert the raw DC matrix and save it as a NIfTI image
        dc_img = array_to_nifti(brain_masker,dc)
        dc_img_name = fmri_list[i][0][:6] + "_wDC_raw"
        nib.save(dc_img, dc_img_name)
        print("Raw wDC image of the whole GM of " + fmri_list[i][0][:6] + " saved.")
        
    print("Done!")

In [ ]:
# Run the analysis
calc_dc_auto("/Volumes/Seagate Dr/PhD/CBD/BOLD_data",
             "/Users/mishodimitrov/Downloads/PhD/Analysis/CBD/Data/cbd_session_list.txt",
             "/Users/mishodimitrov/Downloads/PhD/Analysis/CBD/Data/Masks/GM",
            "final_resampled_GM_mask_3.0mmMNI.nii.gz",
            )